In [23]:
import numpy as np    
import pandas as pd
import os
from tqdm import tqdm_notebook as tqdm

from skimage import data, io
from skimage.transform import resize

from sklearn.model_selection import train_test_split
import shutil


In [57]:
train_features

,filepath,site
id,,
ZJ000000,../data/train_features/ZJ000000.jpg,S0120
ZJ000001,../data/train_features/ZJ000001.jpg,S0069
ZJ000002,../data/train_features/ZJ000002.jpg,S0009
ZJ000003,../data/train_features/ZJ000003.jpg,S0008
ZJ000004,../data/train_features/ZJ000004.jpg,S0036
...,...,...
ZJ016483,../data/train_features/ZJ016483.jpg,S0093
ZJ016484,../data/train_features/ZJ016484.jpg,S0043
ZJ016485,../data/train_features/ZJ016485.jpg,S0089


In [4]:
train_features = pd.read_csv("../data/train_features.csv", index_col="id")
train_labels = pd.read_csv("../data/train_labels.csv", index_col="id")

train_features["filepath"] = train_features["filepath"].apply(
    lambda x: os.path.join("/Users/nicholasabad/Desktop/workspace/driven-data-conservision/data", x)
)

In [ ]:
class DataLoader:
    def __init__(
        self, 
        training_features: pd.DataFrame,
        training_labels: pd.DataFrame,
        validation_split: float=0.2,
        resized_image_shape: tuple=(224, 224),
        random_state: int=42
    ):
        self.training_features = training_features
        self.training_labels = training_labels
        self.validation_split = validation_split
        self.resized_image_shape = resized_image_shape
        self.random_state = random_state

    def _convert_images_to_numpy_array_and_resize(
        self,
        paths_to_images: list,
    ):
        # Helper function to ensure all images are RGB
        def ensure_rgb(image):
            if len(image.shape) == 2:  # Grayscale (H, W)
                return np.stack([image] * 3, axis=-1)  # Convert to (H, W, 3)
            elif len(image.shape) == 3 and image.shape[2] == 3:  # Already RGB
                return image
            else:
                raise ValueError(f"Unexpected image shape: {image.shape}")
            
        return np.stack([
            ensure_rgb(resize(
                io.imread(path),
                output_shape=self.resized_image_shape,
                anti_aliasing=True,
            )) / 255.0
            for path in tqdm(paths_to_images, desc="Loading data into np array:")
        ], axis=0)
        
    def split_data(self):
        # Split the data into a training and validation split.
        y = self.training_labels
        x = self.training_features.loc[y.index].filepath.to_frame()
        
        x_train, x_val, y_train, y_val = train_test_split(
            x, y, stratify=y, test_size=self.validation_split, random_state=self.random_state
        )
        
        self.x_train = self._convert_images_to_numpy_array_and_resize(x_train["filepath"])
        self.x_val = self._convert_images_to_numpy_array_and_resize(x_val["filepath"])
        
        self.y_train = y_train
        self.y_val = y_val
        
        
    
        

In [71]:
data_loader = DataLoader(
    training_features = train_features,
    training_labels = train_labels,
    validation_split = 0.2,
    resized_image_shape = (224, 224),
    random_state = 42
)

data_loader.split_data()

/var/folders/2s/38kg81v526j7qcgd8gdq8jd00000gn/T/ipykernel_13587/2928171840.py:37: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for path in tqdm(paths_to_images, desc="Loading data into np array:")


Loading data into np array::   0%|          | 0/13190 [00:00<?, ?it/s]

: 

In [48]:
train_features

,filepath,site
id,,
ZJ000000,../data/train_features/ZJ000000.jpg,S0120
ZJ000001,../data/train_features/ZJ000001.jpg,S0069
ZJ000002,../data/train_features/ZJ000002.jpg,S0009
ZJ000003,../data/train_features/ZJ000003.jpg,S0008
ZJ000004,../data/train_features/ZJ000004.jpg,S0036
...,...,...
ZJ016483,../data/train_features/ZJ016483.jpg,S0093
ZJ016484,../data/train_features/ZJ016484.jpg,S0043
ZJ016485,../data/train_features/ZJ016485.jpg,S0089


In [13]:
x = train_features.loc[y.index].filepath.to_frame()
x

,filepath
id,
ZJ000625,../data/train_features/ZJ000625.jpg
ZJ008571,../data/train_features/ZJ008571.jpg
ZJ008106,../data/train_features/ZJ008106.jpg
ZJ011175,../data/train_features/ZJ011175.jpg
ZJ003323,../data/train_features/ZJ003323.jpg
...,...
ZJ000177,../data/train_features/ZJ000177.jpg
ZJ007834,../data/train_features/ZJ007834.jpg
ZJ015074,../data/train_features/ZJ015074.jpg


In [35]:
def _convert_images_to_numpy_array(
    paths_to_images: list,
):
    # Helper function to ensure all images are RGB
    def ensure_rgb(image):
        if len(image.shape) == 2:  # Grayscale (H, W)
            return np.stack([image] * 3, axis=-1)  # Convert to (H, W, 3)
        elif len(image.shape) == 3 and image.shape[2] == 3:  # Already RGB
            return image
        else:
            raise ValueError(f"Unexpected image shape: {image.shape}")
        
    return np.stack([
        ensure_rgb(resize(
            io.imread(path),
            output_shape=(224, 224),
            anti_aliasing=True,
        )) / 255.0
        for path in tqdm(paths_to_images, desc="Loading data into np array...")
    ], axis=0)

# With Generator

In [27]:
# Create a train and validate directory with each directory having one of the n classes.

class DirectoryCreation():
    def __init__(
        self, 
        path_to_directory: str,
        training_labels: pd.DataFrame,
        training_features: pd.DataFrame,
        random_seed: int=42, 
        validation_split_size: float=0.2
    ):
        self.path_to_directory = path_to_directory
        self.training_labels = training_labels
        self.training_features = training_features
        self.random_seed = random_seed
        self.validation_split_size = validation_split_size
        
    def create_random_split(self):
        # Split the data into a training and validation split.
        y = self.training_labels
        x = self.training_features.loc[y.index]["filepath"].to_frame()
        
        x_train, x_val, y_train, y_val = train_test_split(
            x, y, stratify=y, test_size=self.validation_split_size, random_state=self.random_seed
        )
        
        self.x_train = x_train
        self.x_val = x_val
        self.y_train = y_train
        self.y_val = y_val
        
    def write_images_to_directory(self):
        path_to_training_images = os.path.join(
            self.path_to_directory,
            "train"
        )
        if os.path.exists(path_to_training_images):
            shutil.rmtree(path_to_training_images)
        os.mkdir(path_to_training_images)
        
        path_to_validation_images = os.path.join(
            self.path_to_directory,
            "validation"
        )
        
        if os.path.exists(path_to_validation_images):
            shutil.rmtree(path_to_validation_images)
        os.mkdir(path_to_validation_images)
            
            
        for column in self.y_train.columns:
            train_class_subfolder = os.path.join(
                path_to_training_images,
                column
            )
            os.mkdir(train_class_subfolder)
            
            validation_class_subfolder = os.path.join(
                path_to_validation_images,
                column
            )
            os.mkdir(validation_class_subfolder)
            
        # Determine where to save each file.
        list_of_categories = list(self.y_train.columns)
        
        for idx, row in tqdm(self.y_train.iterrows(), total = self.y_train.shape[0], desc="Copying training files..."):
            category = list_of_categories[self.y_train.loc[idx].argmax()]
            path_to_original_file = self.x_train.loc[idx, "filepath"]
            
            shutil.copy(
                src=path_to_original_file,
                dst=os.path.join(
                    path_to_training_images,
                    category
                )
            )
            
        for idx, row in tqdm(self.y_val.iterrows(), total = self.y_val.shape[0], desc="Copying validation files..."):
            category = list_of_categories[self.y_val.loc[idx].argmax()]
            path_to_original_file = self.x_val.loc[idx, "filepath"]
            
            shutil.copy(
                src=path_to_original_file,
                dst=os.path.join(
                    path_to_validation_images,
                    category
                )
            )
        
    
directory = DirectoryCreation(
    path_to_directory = "/Users/nicholasabad/Desktop/workspace/driven-data-conservision/data",
    training_labels = train_labels,
    training_features = train_features,
    random_seed = 42,
    validation_split_size = 0.2,
)

directory.create_random_split()
directory.write_images_to_directory()


/var/folders/2s/38kg81v526j7qcgd8gdq8jd00000gn/T/ipykernel_41412/2021609546.py:67: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, row in tqdm(self.y_train.iterrows(), total = self.y_train.shape[0], desc="Copying training files..."):


Copying training files...:   0%|          | 0/13190 [00:00<?, ?it/s]

/var/folders/2s/38kg81v526j7qcgd8gdq8jd00000gn/T/ipykernel_41412/2021609546.py:79: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, row in tqdm(self.y_val.iterrows(), total = self.y_val.shape[0], desc="Copying validation files..."):


Copying validation files...:   0%|          | 0/3298 [00:00<?, ?it/s]

In [30]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Paths to training and validation directories
train_dir = "/Users/nicholasabad/Desktop/workspace/driven-data-conservision/data/train"
val_dir = "/Users/nicholasabad/Desktop/workspace/driven-data-conservision/data/validation"

# Create ImageDataGenerator for training and validation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values to [0, 1]
    rotation_range=20,    # Randomly rotate images
    width_shift_range=0.2,  # Randomly shift horizontally
    height_shift_range=0.2, # Randomly shift vertically
    shear_range=0.2,       # Shear transformation
    zoom_range=0.2,        # Random zoom
    horizontal_flip=True,  # Flip images horizontally
    fill_mode="nearest"    # Fill empty pixels after transformations
)

val_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0  # Normalize pixel values to [0, 1]
)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

val_generator = val_datagen.flow_from_directory(
    directory=val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

Found 13190 images belonging to 8 classes.
Found 3298 images belonging to 8 classes.


In [31]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')  # Match number of classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/Users/nicholasabad/miniconda3/envs/drivendata/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [32]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=10
)

/Users/nicholasabad/miniconda3/envs/drivendata/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
147/412 ━━━━━━━━━━━━━━━━━━━━ 1:50 415ms/step - accuracy: 0.2174 - loss: 2.7647

KeyboardInterrupt: 

In [1]:
import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))

[]
